In [1]:
import pandas as pd #importing Python modules
import numpy as np

**1-install kallisto**

Accessing Kallisto(which has been installed using the command line) - ! is used to run the command from the command line

In [2]:
!kallisto 

kallisto 0.46.0

Usage: kallisto <CMD> [arguments] ..

Where <CMD> can be one of:

    index         Builds a kallisto index 
    quant         Runs the quantification algorithm 
    bus           Generate BUS files for single-cell data 
    pseudo        Runs the pseudoalignment step 
    merge         Merges several batch runs 
    h5dump        Converts HDF5-formatted results to plaintext
    inspect       Inspects and gives information about an index
    version       Prints version information
    cite          Prints citation information

Running kallisto <CMD> without arguments prints usage information for <CMD>



**2-reproduce Moriarty’s result**

Index - generation using the arc.fasta example

Generate a file called 'transcriptome.index' (index from the transcript using arc.fasta) 

In [3]:
!kallisto index -i transcriptome.idx arc.fasta


[build] loading fasta file arc.fasta
[build] k-mer length: 31
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 19 contigs and contains 10000 k-mers 



quant is used to 'Runs the quantification algorithm' - Basically I want to generate the abundance for the reads the using the same input as Moriarty and expect the same output 
output is the name of the folder that we generate(name that I chose) 
--single is used because I'm trying to 'Quantify single-end reads'
the reads are done from a 'cDNA fragment library of mean length 150bp , standard deviation 20bp'.

Therefore I set the :
'Estimated average fragment length' to 150 
'Estimated standard deviation of fragment length 'to  20

arc.fastq are the reads 


In [4]:
!kallisto quant -i transcriptome.idx -o output --single -l 150 -s 20 arc.fastq


[quant] fragment length distribution is truncated gaussian with mean = 150, sd = 20
[index] k-mer length: 31
[index] number of targets: 10
[index] number of k-mers: 10,000
[index] number of equivalence classes: 26
[quant] running in single-end mode
[quant] will process file 1: arc.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 100,000 reads, 99,983 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 58 rounds



processed 100,000 reads, 99,983 reads pseudoaligned - this seems resonable 

The previous line of code generated a folder 'Ouput' that contain 3 different documents :
    _ abundance.h5
    _ abundance.tsv
    _run_info.json
The next step is to read to  abundance.tsv file and check if the tpm for each arc correspond to the TPM obtaiend by Moriarty    

In [5]:
output_data=pd.read_csv('output/abundance.tsv',header= None, index_col= False,sep="\s+")
output_data

,0,1,2,3,4
0,target_id,length,eff_length,est_counts,tpm
1,Arc1,4000,3851,2810.71,20570.4
2,Arc2,2000,1851,3640.85,55436.5
3,Arc3,3000,2851,28233.5,279105
4,Arc4,4000,3851,10376.2,75939.3
5,Arc5,4000,3851,12912.9,94504.2
6,Arc6,3000,2851,1983.17,19604.8
7,Arc7,2000,1851,5442.49,82868.8
8,Arc8,2000,1851,5635.31,85804.8
9,Arc9,3000,2851,3082.56,30472.9


since the results are close but not exactly the same as Moriaty we can conclude that the author rounded the obtained results 



transcriptome example-2 first gene

In [6]:
!gunzip -c test_copy/transcripts.fasta.gz | head -4

>ENST00000513300.5
AGTGCCTTTGGCGCACTGAGGTGCACAGGGTCCCTTAGCCGGGCGCAGGGCGCGCAGCCCAGGCTGAGATCCGCGGCTTCCGTAGAAGTGAGCATGGCTGGGCAGCGAGTGCTTCTTCTAGTGGGCTTCCTTCTCCCTGGGGTCCTGCTCTCAGAGGCTGCCAAAATCCTGACAATATCTACAGTAGATTTTAAAAAGGAAGAAAAATCATATCAAGTTATCAGTTGGCTTGCACCTGAAGATCATCAAAGAGAATTTAAAAAGAGTTTTGATTTCTTTCTGGAAGAAACTTTAGGTGGCAGAGGAAAATTTGAAAACTTATTAAATGTTCTAGAATACTTGGCGTTGCAGTGCAGTCATTTTTTAAATAGAAAGGATATCATGGATTCCTTAAAGAATGAGAACTTCGACATGGTGATAGTTGAAACTTTTGACTACTGTCCTTTCCTGATTGCTGAGAAGCTTGGGAAGCCATTTGTGGCCATTCTTTCCACTTCATTCGGCTCTTTGGAATTTGGGCTACCAATCCCCTTGTCTTATGTTCCAGTATTCCGTTCCTTGCTGACTGATCACATGGACTTCTGGGGCCGAGTGAAGAATTTTCTGATGTTCTTTAGTTTCTGCAGGAGGCAACAGCACATGCAGTCTACATTTGACAACACCATCAAGGAACATTTCACAGAAGGCTCTAGGCCAGTTTTGTCTCATCTTCTACTGAAAGCAGAGTTGTGGTTCATTAACTCTGACTTTGCCTTTGATTTTGCTCGACCTCTGCTTCCCAACACTGTTTATGTTGGAGGCTTGATGGAAAAACCTATTAAACCAGTACCACAAGACTTGGAGAACTTCATTGCCAAGTTTGGGGACTCTGGTTTTGTCCTTGTGACCTTGGGCTCCATGGTGAACACCTGTCAGAATCCGGAAATCTTCAAGGAGATGAACAATGCCTTTGCTCACCTACCCCAAGGGGTGATATGGAAG

2 first genes from transcripts.fasta, I used head -4 because I want to 2 genes and each gene is represented in 2 line 
-One for the name 
-One for the sequence

three reads from reads_1.fastq, I used head -12 because I want 3 reads and each read is represented in 4 lines :

-one for the name

-one for the sequence 

-one with a + 

-one with quality values 


In [7]:
! gunzip -c test_copy/reads_1.fastq.gz | head -12

@1:NM_014620:16:182
GTTCCGAGCGCTCCGCAGAACAGTCCTCCCTGTAAGAGCCTAACCATTGC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@2:NM_014620:1094:172
ATGAAAAAAATTCACGTTAGCACGGTGAACCCCAATTATAACGGAGGGGA
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@3:NM_022658:294:172
TGTACGGGCCCGGCGGCTCGGCGCCCGGCTTCCAGCACGCTTCGCACCAC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
gunzip: error writing to output: Broken pipe
gunzip: test_copy/reads_1.fastq.gz: uncompress failed


outputs 3 reads

# 3 simulate an Arc transcriptome and RNA-seq reads

I breaked this into 3 subparts
- produce the FASTA file 
- generate 1 read and write it into a FASTQ file
- produce the FASTQ file (with 100 000 reads)


# 3.1 produce the FASTA file 

3.1.A- Create a first dictionary dict_segment

create a dictionary called dict_segment
each key is a letter from A to J(corresponding to the 10 different segement) and each value is the corresponding 1000 nt sequence(randomly generated)
Each nucleotide has an equal probablity to be 'A'-'C'-'T'or 'G' therefore proba=[0.25,0.25,0.25,0.25]
The way I implemented it : for loop 

In [8]:
dict_segment= {} #initialisation 
proba=[0.25,0.25,0.25,0.25]
sequence_length=1000
letter=['A','B','C','D','E','F','G','H','I','J']
for i in letter: 
    string_name=(i)
    string_sequence=''.join(np.random.choice(list('ATCG'),sequence_length,p=proba))
    dict_segment[(string_name)] = string_sequence #set the key(letter of the segement) and the value(sequence of that segment)
np.random.seed(1)     


3.1.B- Create a second dictionary for the transcripts 
Each transcripts(arc1..arc10) is composed of a combination of different segements.
3.1.C- Create a fasta file for the trancripts

generation of .fasta file for trancripts: 
The way I did it was to use the values of the dic_segment to extract the corresponding sequence of each transcript(knowing that is it always a combination of different segements(A..J).

In the same for loop I also created a dictionary called fragment_segment
Each key is the trancript name(Arc1 for exemple) and each value is the corresponding sequence(for Arc1 it is the sequence composed of the fragements'A','B','C'&'D')


Please note that i choose to not use modular - why ? because i think using the dictionary composed of segements I can generate transcripts with  more flexibility ( theorically we can generate a transcrpit composed of segment A,C&J for exemple) that would be harder to implement with modular.

I commented the Arc_1 part bellow and all the the other transcripted have been written in the file in a similar way


In [9]:
a=open('transcripts.fasta', 'w') #we will store 
fragment_segment= {} #initialisation 
Arc_1= ('A','B','C','D') #composed of the segments 'A','B','C','D' - will be used to loop over 
string_name=('>'+'Arc1') #format for trancripts name in a fasta file
a.write(string_name) #write >Arc1 to the file 
a.write('\n') #go to the line 
temp_list = [] #temporary list, will be used in the for loop
for i in Arc_1:  #loop over all the segments that are composing Arc_1
    a.write(dict_segment[i]) #write sequence segement i in the transcripts.fasta file (i will take the value of A,B,C,D in this order)
    temp_list.append(dict_segment[i]) #append to the temporary list (stores temporarly the sequence)
    fragment_segment['arc1']= ''.join(temp_list) #set the key to be Arc1& 
    #assigns the value to be the sequence 
    #''.join is used of to remove the ','
a.write('\n')    #go to the line before the next transcript 



Arc_2= ('B','C')
string_name=('>'+'Arc2')
a.write(string_name)
a.write('\n')  
sequence_Arc_2=() #initialize 
temp_list = [] #reinitialize the list 
for i in Arc_2:
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc2']= ''.join(temp_list)
a.write('\n')     
    
Arc_3= ('C','D','E')
sequence_Arc_3=()
string_name=('>'+'Arc3')
a.write(string_name)
a.write('\n') 
temp_list = [] #reinitialize the list 
for i in Arc_3: 
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc3']= ''.join(temp_list)
a.write('\n') 
    
Arc_4= ('D','E','F','G')
sequence_Arc_4=()
string_name=('>'+'Arc4')
a.write(string_name)
a.write('\n') 
temp_list = []
for i in Arc_4:  
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc4']= ''.join(temp_list)
a.write('\n')        
    
Arc_5= ('E','F','G','H')
sequence_Arc_5=()
string_name=('>'+'Arc5')
a.write(string_name)
a.write('\n') 
temp_list = []
for i in Arc_5:  
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc5']= ''.join(temp_list)
a.write('\n')    
    
Arc_6= ('F','G','H')
sequence_Arc_6=()
string_name=('>'+'Arc6')
a.write(string_name)
a.write('\n') 
temp_list = []
for i in Arc_6: 
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc6']= ''.join(temp_list)
a.write('\n') 

Arc_7= ('G','H')
sequence_Arc_7=()
string_name=('>'+'Arc7')
a.write(string_name)
a.write('\n') 
temp_list = []
for i in Arc_7: 
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc7']= ''.join(temp_list)
a.write('\n') 

    
Arc_8= ('H','I')
sequence_Arc_8=()
string_name=('>'+'Arc8')
a.write(string_name)
a.write('\n')
temp_list = []
for i in Arc_8:
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc8']= ''.join(temp_list)
a.write('\n')
  
    
Arc_9= ('I','J','A')
sequence_Arc_9=()
string_name=('>'+'Arc9')
a.write(string_name)
a.write('\n')
temp_list = []
for i in Arc_9:
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc9']= ''.join(temp_list)
a.write('\n')
     
    
Arc_10= ('J','A','B')
sequence_Arc_10=()
string_name=('>'+'Arc10')
a.write(string_name)
a.write('\n')
temp_list = []
for i in Arc_10:
    a.write(dict_segment[i])
    temp_list.append(dict_segment[i])
    fragment_segment['arc10']= ''.join(temp_list)
a.write('\n')
a.close()

At this point we generated the FASTA file 

# 3.2 generate 1 read and write it into a FASTQ file

the final FASTQ file will be composed of 100 000 reads- but I chose to first generate 1 single read and then create a function that include a for loop to generate 100 000k reads.

+ 3.2.1 select a sample a transcript i according to its nucleotide abundance νi

pick one of 10 transcripts based on abundances
picked_transcript is the randomly picked trancrips( on of the 10 arcs)
picked_sequence is its corresponding sequence(accessed using the dictionary created in 3.1B 


In [10]:
abundances=[0.008,0.039,0.291,0.112,0.127,0.008,0.059,0.060,0.022,0.273] #extracted from the table
len(abundances) #check that we have 10 abundances (1 for each trancript)
sum(abundances) #check - the probablity dont add to 1 - need to normalize 
normalised_factor = sum(abundances)
abundances_normalised_transcripts = [x / normalised_factor for x in abundances]
print('the sum of the nomralised abundances is',sum(abundances_normalised_transcripts))  #check - this now sum up to 1.
arcs=['arc1','arc2','arc3','arc4','arc5','arc6','arc7','arc8','arc9','arc10'] #possible randoms"picks"
#np.random.seed(1) #used for reproducibility
picked_transcript=np.random.choice(arcs,p=abundances_normalised_transcripts) #peaks one of the arc using the abundances

print('the selected transcript is',picked_transcript) #test
picked_sequence=fragment_segment[picked_transcript] #sequence of the transcript selected out of the 10 transcripts
print('its corresponding sequence is   :',picked_sequence)



the sum of the nomralised abundances is 1.0
the selected transcript is arc4
its corresponding sequence is   : TGGAAATAGCTCTGACTACAGACCGCAGAAATGTCAATTTGTTCCTCGGGATTTTAGCATCCCTCCGTATTCTGTCCACCACCCCTCGACCTAAGCGGGGGGGCAAATTGGGGCGTGCCCATATACTCAGTCTTGTCGACCTAGCCAGCGTAATTGATCAGGCAATTTGACGATGCTACAACTTCGGGTACGGAAACCTACTCACACGCGAACATGAATGTCAATTCTAAATTAACAGGTTCAACGCGACACTTATTTGGCGAGTTTGTGAACGCGAGCGGGCTGTACTACGCTTCACAGCAGGTTCGCAGAATACAAAACGCGGGCTCCATAAGCTGAAAACGCAGGCTGCATCTACGTTTCCCGCTGACCGCTTCACGTATATTGAGTAACTAGATTTTAAAGATATAAAGGTCTCAGCCCGTTAGATGAGTCTAATCAGAAAGAGTCAGACTGCCGACTAGCAAGTCACGTGCGTTATTGTTATGTCGGATAATACGTCGAGGGTATATGACAGGCATCTCTTCACATGGGCTAATCCCACTTCACCAGTTTTTTACGGAGCACTAATCTTAGTCACAAAAGCTGCAAAATTACGAATATTCAAAACACGCGACCTTAAATCGGCGAGGTATAGTTCAGGGAGGCAATATGTCACCAGGTGAGTGGGGGTGACACAACAAGGACAGAGCTCTAGTTAGGCCGTGTCGAAGTGTCTGCCCGAAATAATCTATGACACCACGGGTATAATAAAAGGGCTTTCCATCTACTCGCCAACTCGAAGAGCCAACCGGAACCCTCTCTGGTGGAGATCCCGGCTTGGTATCAAGGGGAGCAATCGGCCATTCAAGGCGAAGCTGTAGGAATTGGAATTCTTCACGACCCGGTAA

+ 3.2.2 pick a random fragment length at least as long as one read from a truncated Gaussian of mean length 150, standard deviation 20 (truncated because: no shorter than a read, and no longer than the whole transcript), 
please note that for this part I mainly used the code provided in the homework

In [11]:
L_picked_sequence=len(picked_sequence)
mean_frag=150
sd_frag=20
len_R=75
#np.random.seed(2)
while True:
    fraglen = int(np.random.normal(mean_frag, sd_frag))
    if fraglen >= len_R: break
if fraglen > L_picked_sequence: fraglen = L_picked_sequence
print('the random length is ', fraglen,'long')

the random length is  133 long


- 3.2.3 pick a random fragment of that length from transcript i;

upper limit of what we can use as my index to select the random fragement 
I will refer to Random fragment length as RFL



In [12]:
upper_limit_index=len(picked_sequence)-fraglen+1
#np.random.seed(3)
start_index=np.random.randint(0,upper_limit_index)#generate an int that can take the maximum value of upper_limit_index
end_index=start_index+fraglen
#print(start_index)
#print(end_index)

RFL_picked_sequence=picked_sequence[start_index:end_index]
print('the random fragement selected out of the sequence transcript is', RFL_picked_sequence)


the random fragement selected out of the sequence transcript is CCAGCGTAATTGATCAGGCAATTTGACGATGCTACAACTTCGGGTACGGAAACCTACTCACACGCGAACATGAATGTCAATTCTAAATTAACAGGTTCAACGCGACACTTATTTGGCGAGTTTGTGAACGCGA


- 3.2.4 pick a 75nt read by choosing a random orientation, taking the read from one of the two ends of the fragment. If on the top strand, your read is the first 75nt of the fragment. If on the bottom strand, your read is the last 75nt of the fragment, reverse complemented.

the orientation is either forward or reverse the 2 senario happen with equal probablity - 
so the function np.random.choice(oritentations_possibilities,p=proba_orientation) will return forward or reverse with probablity 1/2

if the orientation is forward we just have to read the first 75 nt - stored into a variable called selected_75nt

if the is reverse we have to do several step 
1)select the last 75nt 
2)reverse the order (ex if the sequence is ATCG the reverse ordered one will be GCTA(like reading from right to left)
3)translate the reversed sequence - A replaced by T , and G replaced by C (and visversa)
T=str.maketrans('ATCG','TAGC') is set the translation that I want to make 
.translate(T) is used to apply the translation to the sequence of interest


the output of 3) is stored into a variable called selected_75nt


In [13]:
proba_orientation=[0.5,0.5]
oritentations_possibilities=['forward','reverse']
T=str.maketrans('ATCG','TAGC')
oritentation=np.random.choice(oritentations_possibilities,p=proba_orientation)
print(oritentation) #check
if oritentation=='forward': 
    selected_75nt = RFL_picked_sequence[0:75] #first 75 nt
    print(selected_75nt)
elif oritentation=='reverse': 
    reverse_orientation_75nt=(RFL_picked_sequence[-75:]) #last 75 nt
    print('the end read seq-normal order')
    print(reverse_orientation_75nt) #check
    reverse_orientation_75ntreverse=(list(reversed(reverse_orientation_75nt))) #reverse order 
    print('the end read seq-reverse order')
    print(reverse_orientation_75ntreverse) #check
    
    selected_75nt=''.join(reverse_orientation_75ntreverse).translate(T) #'',jointo input a string instead of list
    print('the reversed translated read seq')
    print(selected_75nt)
 

forward
CCAGCGTAATTGATCAGGCAATTTGACGATGCTACAACTTCGGGTACGGAAACCTACTCACACGCGAACATGAAT


- 3.2.5 generate a simulated read sequence by adding simulated base calling errors to the 75nt read: given base
calling accuracy , at each base, with probability(1-Alpha).

the way I implemented it - using a for loop to go through all the nucleotides that compose selected_75nt
for each one, i called an np.random.choice that will set read_i to error with a probablity of 1-0.999 ( so 0.001)
so in most of the cases nothing will happen but when the read_i is error we have to switch this nucleotide base with one of the 3 others 
Say for exemple that the nucleotide for which I need to generate an error is A this nucleotide can only be replaced by T,C or G but NOT A. And this nucleotide has to be replaced with T C G with equal probablities(so 1/3 for each of the 3 bases?
How did i implemented that - first i created set_possibility which contains all four nucleotides and I substracted from it the nucelotide that need to be replaced (set(set_possibility) - set(i))

and then passed a np.random.choice with arguments replacement_possibilities as outcome and p=[1/3,1/3,1/3].



In [14]:
set_possibility=['A','T','C','G']
alpha=0.999

for i in selected_75nt :
    read_i=np.random.choice([i,'error'],p=[alpha,1-alpha])
    if read_i == 'error': #if read_i is error it means that the base has to be changed 
        replacement_possibilities= set(set_possibility) - set(i) #it can only be changed by a base that is different from the one it is 
        i=np.random.choice(list(replacement_possibilities),p=[1/3,1/3,1/3])#all other bases have equal probablities to be selected
print(selected_75nt)    
    

CCAGCGTAATTGATCAGGCAATTTGACGATGCTACAACTTCGGGTACGGAAACCTACTCACACGCGAACATGAAT


Now that I generated 1 random read I want to write it into a file using the fastq format 

In [15]:
b=open('test.fastq', 'w') #creation of a file called test.fatq
read_name='read1' 
b.write('@:'+read_name)
b.write('\n')
b.write(selected_75nt) 
b.write('\n')
b.write('+')
b.write('\n')
quality_values= "I" * len(selected_75nt) #prints 75 I's
b.write(quality_values)
b.close()

# 3.3 produce the FASTQ file (with 100 000 reads)


To do this part I basically used the code that I generated in the 3.2 part and just added a four loop 
I re-wrote the constants used (that have been defined progressively in 3.2), just to be more organised.

In [16]:
#constants- dont change for all the 1K reads 
abundances=[0.008,0.039,0.291,0.112,0.127,0.008,0.059,0.060,0.022,0.273] 
normalised_factor = sum(abundances)
abundances_normalised_transcripts= [x / normalised_factor for x in abundances]
arcs=['arc1','arc2','arc3','arc4','arc5','arc6','arc7','arc8','arc9','arc10']
mean_frag=150
sd_frag=20
len_R=75
set_possibility=['A','T','C','G']
alpha=0.999
proba_orientation=[0.5,0.5]
oritentations_possibilities=['forward','reverse']
T=str.maketrans('ATCG','TAGC')


the function called read_function takes 4 arguments 
- nb_of_reads - the number of reads we want to generate
- file_fastq - file in which we want to write the reads generated
- dict_transcripts -that correspond to different possible transcripts (fragment_segment)
- abundances_normalised- and the abundances associated with each one of the trancripts 


The final goal of this fucntion is to write the FASTQ file generating the reads using a list of possible transcripts and the abundances associated with each of those

In [17]:
def read_function(nb_of_reads,file_fastq,dict_transcripts,abundances_normalised):
    for read_rep in range(0,nb_of_reads): 
        
        np.random.seed(read_rep) #make sure to generate a new sequence everytime, use read_rep as an argument of np.random.seed()- but used for reproducibility
        picked_transcript=np.random.choice(arcs,p=abundances_normalised)
        picked_sequence=dict_transcripts[picked_transcript] #sequence of the transcript selected out of the 10 transcripts
    #depending on the abundances
        L_picked_sequence=len(picked_sequence)
    
        while True:
            fraglen = int(np.random.normal(mean_frag, sd_frag))
            if fraglen >= len_R: break
        
    
   
    
        upper_limit_index=len(picked_sequence)-fraglen
        start_index=np.random.randint(0,upper_limit_index)#generate an int that can take the maximum value of upper_limit_index
        end_index=start_index+fraglen    
        RFL_picked_sequence=picked_sequence[start_index:end_index]#random fragment of that length from transcript i
    
    
        oritentation=np.random.choice(oritentations_possibilities,p=proba_orientation)
    
        if oritentation=='forward': 
             selected_75nt = RFL_picked_sequence[0:75] #first 75 nt
        elif oritentation=='reverse': 
            reverse_orientation_75nt=(RFL_picked_sequence[-75:]) #last 75 nt
            reverse_orientation_75ntreverse=(list(reversed(reverse_orientation_75nt)))
            selected_75nt=''.join(reverse_orientation_75ntreverse).translate(T)
  
      
        for i in selected_75nt :
            read_i=np.random.choice([i,'error'],p=[alpha,1-alpha])
            if read_i == 'error': #if read_i is error it means that the base has to be changed 
                replacement_possibilities= set(set_possibility) - set(i) #it can only be changed by a base that is different from the one it is 
            
                i=np.random.choice(list(replacement_possibilities),p=[1/3,1/3,1/3])
            
    #writing in the file  
            
        read_name='@:read'+str(read_rep+1)  #so no 0 
        file_fastq.write(read_name)
        file_fastq.write('\n')
        file_fastq.write(selected_75nt)
        file_fastq.write('\n')
        file_fastq.write('+')
        file_fastq.write('\n')
        quality_values= "I" * 75
        file_fastq.write(quality_values)
        file_fastq.write('\n')
 

calling the function read_function- the 100 000 reads will be outputed in the readL.fastq file 

In [18]:
c=open('readL.fastq', 'w') #creation of a file 
read_function(nb_of_reads=100000,file_fastq=c,dict_transcripts=fragment_segment,abundances_normalised=abundances_normalised_transcripts)

# 4 test kallisto

In [19]:
!kallisto index -i transcriptome_2.idx transcripts.fasta #generating the index using the transcripts.fasta file I created


[build] loading fasta file transcripts.fasta
[build] k-mer length: 31
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 19 contigs and contains 10000 k-mers 



In [20]:
!kallisto quant -i transcriptome_2.idx -o output_2 --single -l 150 -s 20 readL.fastq


[quant] fragment length distribution is truncated gaussian with mean = 150, sd = 20
[index] k-mer length: 31
[index] number of targets: 10
[index] number of k-mers: 10,000
[index] number of equivalence classes: 26
[quant] running in single-end mode
[quant] will process file 1: readL.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 99,992 reads, 99,992 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 57 rounds



In [21]:
expected_TPM=['expected_tpm',6000,58000,290000,83000,94000,7800,87000,88000,22000,270000]
moriarty_results=['moriarty',21000,55000,280000,76000,95000,20000,83000,86000,30000,260000]
output2_data=pd.read_csv('output_2/abundance.tsv',header= None, index_col= False,sep="\s+")
output2_data['moriarty_results']=moriarty_results
output2_data['expected_TPM']=expected_TPM

output2_data

,0,1,2,3,4,moriarty_results,expected_TPM
0,target_id,length,eff_length,est_counts,tpm,moriarty,expected_tpm
1,Arc1,4000,3851,2939.2,21500.9,21000,6000
2,Arc2,2000,1851,3666.61,55803.4,55000,58000
3,Arc3,3000,2851,28313.6,279769,280000,290000
4,Arc4,4000,3851,10521.8,76969.4,76000,83000
5,Arc5,4000,3851,12700,92903.4,95000,94000
6,Arc6,3000,2851,1823.37,18016.9,20000,7800
7,Arc7,2000,1851,5468.69,83229.7,83000,87000
8,Arc8,2000,1851,5681.56,86469.5,86000,88000
9,Arc9,3000,2851,3163.21,31255.9,30000,22000


# 5 Debug Kalisto

From the table bellow showed the the tpm obtained are very close to the results obtained by Moriarty but not close to the expected ones. 
I believe that the issue is that kallisto dont take as a hypothesis the overlap. 
To test I will first create a new fasta file composed of transcripts wtih no overlap 
arc1-segmentA 
ARC2-segmentB 
etc 
very similar to the process of what i used before to generate a transcript fasta file and dictionary corresponding to transcripts  


In [22]:
#creation of another fasta and dictionatry called fragment_segment_debug
#each transcript is composed of 1 segment only(so no possible overlap)
d=open('debug.fasta', 'w') #creation of a file 
proba=[0.25,0.25,0.25,0.25]
sequence_length=1000
fragment_segment_debug= {} #initialisation 


Arc_1_debug= ('A')
string_name=('>'+'Arc1')
d.write(string_name)
d.write('\n')
d.write(dict_segment['A']) 
fragment_segment_debug['arc1']=dict_segment['A']
d.write('\n')   

Arc_2_debug= ('B')
string_name=('>'+'Arc2')
d.write(string_name)
d.write('\n')
d.write(dict_segment['B']) 
fragment_segment_debug['arc2']=dict_segment['B']
d.write('\n')
  
Arc_3_debug= ('C')
string_name=('>'+'Arc3')
d.write(string_name)
d.write('\n')
d.write(dict_segment['C']) 
fragment_segment_debug['arc3']=dict_segment['C']
d.write('\n')


Arc_4_debug= ('D')
string_name=('>'+'Arc4')
d.write(string_name)
d.write('\n')
d.write(dict_segment['D']) 
fragment_segment_debug['arc4']=dict_segment['D']
d.write('\n')
        

Arc_5_debug= ('D')
string_name=('>'+'Arc5')
d.write(string_name)
d.write('\n')
d.write(dict_segment['E']) 
fragment_segment_debug['arc5']=dict_segment['E']
d.write('\n')
        
    
Arc_6_debug= ('F')
string_name=('>'+'Arc6')
d.write(string_name)
d.write('\n')
d.write(dict_segment['F']) 
fragment_segment_debug['arc6']=dict_segment['F']
d.write('\n')    
        
    
Arc_7_debug= ('G')
string_name=('>'+'Arc7')
d.write(string_name)
d.write('\n')
d.write(dict_segment['G']) 
fragment_segment_debug['arc7']=dict_segment['G']
d.write('\n')    
            
    
    
Arc_8_debug= ('H')
string_name=('>'+'Arc8')
d.write(string_name)
d.write('\n')
d.write(dict_segment['H']) 
fragment_segment_debug['arc8']=dict_segment['H']
d.write('\n')    
               
    
Arc_9_debug= ('I')
string_name=('>'+'Arc9')
d.write(string_name)
d.write('\n')
d.write(dict_segment['I']) 
fragment_segment_debug['arc9']=dict_segment['I']
d.write('\n')    
                
    
Arc_10_debug= ('J')
string_name=('>'+'Arc10')
d.write(string_name)
d.write('\n')
d.write(dict_segment['J']) 
fragment_segment_debug['arc10']=dict_segment['J']
d.write('\n')    
                      
d.close()   

to test my result I chose to set the abundances of the 5 first transcripts to be the same and half of the abundances for the 5 last transcripts 
If the 'bug' from kallisto is due to the overlap, we will obtain similar tpm for the arcs from 1 to 5 which is half of the tpm for the arcs 6 to 10 

In [23]:
abundances_debug=[1,1,1,1,1,2,2,2,2,2] 
normalised_factor_debug = sum(abundances_debug)
abundances_normalised_transcripts_debug= [x / normalised_factor_debug for x in abundances_debug]
debug_fastq_file=open('read_debug.fastq', 'w') #creation of a file 
#CALL the function created above using with different arugments  
read_function(nb_of_reads=100000,file_fastq=debug_fastq_file,dict_transcripts=fragment_segment_debug,abundances_normalised=abundances_normalised_transcripts_debug)

In [24]:
!kallisto index -i transcriptome_3.idx debug.fasta


[build] loading fasta file debug.fasta
[build] k-mer length: 31
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 10 contigs and contains 9700 k-mers 



In [25]:
!kallisto quant -i transcriptome_3.idx -o output_3 --single -l 150 -s 20 read_debug.fastq


[quant] fragment length distribution is truncated gaussian with mean = 150, sd = 20
[index] k-mer length: 31
[index] number of targets: 10
[index] number of k-mers: 9,700
[index] number of equivalence classes: 10
[quant] running in single-end mode
[quant] will process file 1: read_debug.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 99,992 reads, 99,065 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 52 rounds



In [26]:
output3_data=pd.read_csv('output_3/abundance.tsv',header= None, index_col= False,sep="\s+")
output3_data

,0,1,2,3,4
0,target_id,length,eff_length,est_counts,tpm
1,Arc1,1000,851,6770,68339
2,Arc2,1000,851,6547,66087.9
3,Arc3,1000,851,6534,65956.7
4,Arc4,1000,851,6520,65815.4
5,Arc5,1000,851,6546,66077.8
6,Arc6,1000,851,13256,133811
7,Arc7,1000,851,13169,132933
8,Arc8,1000,851,13358,134841
9,Arc9,1000,851,13207,133317


the final results obtained is what I would expect - this confirms that the reason Kalisto is messing up with the arcs is because of the overlaps.